In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # default is ‘last_expr’

%load_ext autoreload
%autoreload 2

import json
import os
from random import shuffle
import math
from shapely import wkt

import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter, defaultdict
import numpy as np

In [2]:
%%time

xbd_root = '../../data/'
#all_disasters =['guatemala-volcano', 'hurricane-florence', 'hurricane-harvey', 'hurricane-matthew', 'hurricane-michael', 'mexico-earthquake', 'midwest-flooding', 'palu-tsunami', 'santa-rosa-wildfire', 'socal-fire', 'joplin-tornado', 'lower-puna-volcano', 'moore-tornado', 'nepal-flooding', 'pinery-bushfire', 'portugal-wildfire', 'sunda-tsunami', 'tuscaloosa-tornado', 'woolsey-fire']
all_disasters = ['hurricane-harvey']
label_dirs = [
    xbd_root + 'raw/train/labels',
    xbd_root + 'raw/test/labels',
    xbd_root + 'raw/hold/labels',
    xbd_root + 'raw/tier3/labels'
]
all_files = defaultdict(list)  # wind disaster to list of files (no extension)

all_disasters_tup = tuple(all_disasters)

for label_dir in label_dirs:
    for p in tqdm(os.listdir(label_dir)):
        
        if not p.startswith(all_disasters_tup):
            continue
        
        if not p.endswith('_post_disaster.json'):
            continue
        
        full_path = label_dir+'/'+p
        rel_path = full_path.split(xbd_root)[1]
        
        # example: hurricane-matthew_00000000_post_disaster.json
        disaster_name = p.split('_')[0]
        file = rel_path.split('_post_disaster.json')[0]
        
        all_files[disaster_name].append(file)

100%|██████████| 12738/12738 [00:00<00:00, 1819288.47it/s]

Wall time: 2.51 s


In [3]:
for disaster_name, files in all_files.items():
    print(f'{disaster_name}, {len(files)}')

hurricane-harvey, 522


In [4]:
other_splits = defaultdict(dict)  # disasters other than the wind disasters to train/val/test

for disaster_name, files in all_files.items():

    
    shuffle(files)
    
    num_train_tiles = math.ceil(0.8 * len(files))
    num_val_tiles = math.ceil(0.1 * len(files))
    
    other_splits[disaster_name]['train'] = sorted(files[:num_train_tiles])
    other_splits[disaster_name]['val'] = sorted(files[num_train_tiles:num_train_tiles + num_val_tiles])
    other_splits[disaster_name]['test'] = sorted(files[num_train_tiles + num_val_tiles:])

    print(f"{disaster_name}, train {len(other_splits[disaster_name]['train'])}, val {len(other_splits[disaster_name]['val'])}")

hurricane-harvey, train 418, val 53


In [5]:
with open('../constants/splits/final_mdl_all_disaster_splits.json', 'w') as f:
    json.dump(other_splits, f, indent=4)